# Classification in PySpark's MLlib Project Solution

### Genre classification
Now it's time to leverage what we learned in the lectures to a REAL classification project! Have you ever wondered what makes us, humans, able to tell apart two songs of different genres? How we do we inherenly know the difference between a pop song and heavy metal? This type of classifcation may seem easy for us, but it's a very difficult challenge for a computer to do. So the question is, could an automatic genre classifcation model be possible? 

For this project we will be classifying songs based on a number of characteristics into a set of 23 electronic genres. This technology could be used by an application like Pandora to recommend songs to users or just create meaningful channels. Super fun!

### Dataset
*beatsdataset.csv*
Each row is an electronic music song. The dataset contains 100 song for each genre among 23 electronic music genres, they were the top (100) songs of their genres on November 2016. The 71 columns are audio features extracted of a two random minutes sample of the file audio. These features have been extracted using pyAudioAnalysis (https://github.com/tyiannak/pyAudioAnalysis).

### Your task
Create an algorithm that classifies songs into the 23 genres provided. Test out several different models and select the highest performing one. Also play around with feature selection methods and finally try to make a recommendation to a user.  

For the feature selection aspect of this project, you may need to get a bit creative if you want to select features from a non-tree algorithm. I did not go over this aspect of PySpark intentionally in the previous lectures to give you chance to get used to researching the PySpark documentation page. Here is the link to the Feature Selectors section of the documentation that just might come in handy: https://spark.apache.org/docs/latest/ml-features.html#feature-selectors

Good luck! Have fun :)

### Source
https://www.kaggle.com/caparrini/beatsdataset

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('ClassificationProject').getOrCreate()



In [2]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import * 
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import MinMaxScaler

In [3]:
path ="Datasets/"
df = spark.read.csv(path+'beatsdataset.csv',inferSchema=True,header=True)

In [4]:
df.limit(5).toPandas()

,_c0,1-ZCRm,2-Energym,3-EnergyEntropym,4-SpectralCentroidm,5-SpectralSpreadm,6-SpectralEntropym,7-SpectralFluxm,8-SpectralRolloffm,9-MFCCs1m,...,63-ChromaVector8std,64-ChromaVector9std,65-ChromaVector10std,66-ChromaVector11std,67-ChromaVector12std,68-ChromaDeviationstd,69-BPM,70-BPMconf,71-BPMessentia,class
0,0,0.136440,0.088861,3.201201,0.262825,0.249212,1.114423,0.007003,0.256682,-22.723259,...,0.003431,0.004981,0.010818,0.024001,0.005201,0.015056,133.333333,0.132792,128.0,BigRoom
1,1,0.117039,0.108389,3.194001,0.247657,0.250288,1.065668,0.005387,0.199821,-21.775871,...,0.004461,0.006441,0.007469,0.015499,0.005589,0.019339,120.000000,0.112767,126.0,BigRoom
2,2,0.085308,0.128525,3.123837,0.217205,0.228652,0.789647,0.008247,0.156822,-22.472722,...,0.001529,0.004556,0.007723,0.017482,0.002901,0.022201,133.333333,0.123373,129.0,BigRoom
3,3,0.103050,0.167042,3.150830,0.233593,0.245032,0.967082,0.006571,0.168083,-21.470751,...,0.001591,0.003514,0.009477,0.023162,0.004165,0.015379,133.333333,0.158876,129.0,BigRoom
4,4,0.151730,0.148405,3.194498,0.293730,0.267231,1.353005,0.003872,0.292055,-21.371157,...,0.003945,0.004131,0.011330,0.028188,0.002639,0.019079,133.333333,0.190708,129.0,BigRoom


In [5]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- 1-ZCRm: double (nullable = true)
 |-- 2-Energym: double (nullable = true)
 |-- 3-EnergyEntropym: double (nullable = true)
 |-- 4-SpectralCentroidm: double (nullable = true)
 |-- 5-SpectralSpreadm: double (nullable = true)
 |-- 6-SpectralEntropym: double (nullable = true)
 |-- 7-SpectralFluxm: double (nullable = true)
 |-- 8-SpectralRolloffm: double (nullable = true)
 |-- 9-MFCCs1m: double (nullable = true)
 |-- 10-MFCCs2m: double (nullable = true)
 |-- 11-MFCCs3m: double (nullable = true)
 |-- 12-MFCCs4m: double (nullable = true)
 |-- 13-MFCCs5m: double (nullable = true)
 |-- 14-MFCCs6m: double (nullable = true)
 |-- 15-MFCCs7m: double (nullable = true)
 |-- 16-MFCCs8m: double (nullable = true)
 |-- 17-MFCCs9m: double (nullable = true)
 |-- 18-MFCCs10m: double (nullable = true)
 |-- 19-MFCCs11m: double (nullable = true)
 |-- 20-MFCCs12m: double (nullable = true)
 |-- 21-MFCCs13m: double (nullable = true)
 |-- 22-ChromaVector1m: double (null

In [6]:
df.groupBy('class').count().show(100)

+--------------------+-----+
|               class|count|
+--------------------+-----+
|           PsyTrance|  100|
|           HardDance|  100|
|              Breaks|  100|
|  HardcoreHardTechno|  100|
|   IndieDanceNuDisco|  100|
|              Trance|  100|
|           DeepHouse|  100|
|ElectronicaDowntempo|  100|
|           ReggaeDub|  100|
|             Minimal|  100|
|         DrumAndBass|  100|
|             Dubstep|  100|
|             BigRoom|  100|
|              Techno|  100|
|               House|  100|
|         FutureHouse|  100|
|        ElectroHouse|  100|
|           GlitchHop|  100|
|           TechHouse|  100|
|              HipHop|  100|
|           FunkRAndB|  100|
|               Dance|  100|
|    ProgressiveHouse|  100|
+--------------------+-----+



## Format Data

In [7]:
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+------+---------+----------------+-------------------+-----------------+------------------+---------------+------------------+---------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+-------------------+---------+------------+-------------------+----------------------+--------------------+---------------------+------------------+---------------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+-------------+-------------+-------------+-------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-

In [8]:
input_columns = df.columns # Collect the column names as a list
input_columns = input_columns[1:-1] # keep only relevant columns: from column 1 to 

dependent_var = 'class'

In [9]:
renamed = df.withColumn('label_str', df[dependent_var].cast(StringType()))
indexer = StringIndexer(inputCol='label_str', outputCol='label')
indexed = indexer.fit(renamed).transform(renamed)

In [10]:
numeric_inputs = []
string_inputs = []
for column in input_columns:
    if str(indexed.schema[column].dataType) == 'StringType':
        indexer = StringIndexer(inputCol = column, outputCol=column+'_num')
        indexed = indexer.fit(indexed).transform(indexed)
        new_col_name = column+'_num'
        string_inputs.append(new_col_name)
    else:
        numeric_inputs.append(column)
indexed.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- 1-ZCRm: double (nullable = true)
 |-- 2-Energym: double (nullable = true)
 |-- 3-EnergyEntropym: double (nullable = true)
 |-- 4-SpectralCentroidm: double (nullable = true)
 |-- 5-SpectralSpreadm: double (nullable = true)
 |-- 6-SpectralEntropym: double (nullable = true)
 |-- 7-SpectralFluxm: double (nullable = true)
 |-- 8-SpectralRolloffm: double (nullable = true)
 |-- 9-MFCCs1m: double (nullable = true)
 |-- 10-MFCCs2m: double (nullable = true)
 |-- 11-MFCCs3m: double (nullable = true)
 |-- 12-MFCCs4m: double (nullable = true)
 |-- 13-MFCCs5m: double (nullable = true)
 |-- 14-MFCCs6m: double (nullable = true)
 |-- 15-MFCCs7m: double (nullable = true)
 |-- 16-MFCCs8m: double (nullable = true)
 |-- 17-MFCCs9m: double (nullable = true)
 |-- 18-MFCCs10m: double (nullable = true)
 |-- 19-MFCCs11m: double (nullable = true)
 |-- 20-MFCCs12m: double (nullable = true)
 |-- 21-MFCCs13m: double (nullable = true)
 |-- 22-ChromaVector1m: double (null

#### Skewness

In [11]:
d= {}
for col in numeric_inputs:
    d[col]=indexed.approxQuantile(col,[0.01, 0.99], 0.25)

for col in numeric_inputs:
    skew = indexed.agg(skewness(indexed[col])).collect()
    skew=skew[0][0]
    if skew > 1:
        indexed = indexed.withColumn(col, \
                                    log(when(df[col] < d[col][0], d[col][0])\
                                       .when(indexed[col]> d[col][1], d[col][1])\
                                       .otherwise(indexed[col])+1).alias(col))
        print(col, " has been treated for pos skew", skew)
    elif skew < -1:
        indexed = indexed.withColumn(col, \
                                    exp(when(df[col] < d[col][0], d[col][0])\
                                       .when(indexed[col]> d[col][1], d[col][1])\
                                       .otherwise(indexed[col])+1).alias(col))
        print(col, " has been treated for neg skew", skew)

7-SpectralFluxm  has been treated for pos skew 1.6396138160129063
22-ChromaVector1m  has been treated for pos skew 2.4162415204309258
23-ChromaVector2m  has been treated for pos skew 4.154796693680583
24-ChromaVector3m  has been treated for pos skew 1.1974019617504328
25-ChromaVector4m  has been treated for pos skew 2.446635863594906
26-ChromaVector5m  has been treated for pos skew 2.154482876187508
27-ChromaVector6m  has been treated for pos skew 2.01234064472543
28-ChromaVector7m  has been treated for pos skew 1.1829228989215521
29-ChromaVector8m  has been treated for pos skew 3.7372643733999955
30-ChromaVector9m  has been treated for pos skew 2.4117416421548645
31-ChromaVector10m  has been treated for pos skew 2.1979538518563233
32-ChromaVector11m  has been treated for pos skew 2.1924295373960554
33-ChromaVector12m  has been treated for pos skew 2.278981912155668
41-SpectralFluxstd  has been treated for pos skew 1.8577721462401056
56-ChromaVector1std  has been treated for pos skew 1

In [12]:
minimums = df.select([min(c).alias(c) for c in df.columns if c in numeric_inputs])
min_array = minimums.select(array(numeric_inputs).alias('mins'))
df_minimum = min_array.select(array_min(min_array.mins)).collect()
df_minimum = df_minimum[0][0]
df_minimum

-30.3789543716

In [13]:
features_list = numeric_inputs + string_inputs
assembler = VectorAssembler(inputCols=features_list, outputCol='features')
output = assembler.transform(indexed).select('features','label')


In [14]:
output.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.136439587512,0...|  0.0|
|[0.117038518483,0...|  0.0|
|[0.0853077737447,...|  0.0|
|[0.103049917216,0...|  0.0|
|[0.151729948738,0...|  0.0|
|[0.127046737192,0...|  0.0|
|[0.123395302003,0...|  0.0|
|[0.140027382431,0...|  0.0|
|[0.117635200751,0...|  0.0|
|[0.137400181488,0...|  0.0|
|[0.148838734199,0...|  0.0|
|[0.119846749928,0...|  0.0|
|[0.0786813648231,...|  0.0|
|[0.138335144235,0...|  0.0|
|[0.101304207661,0...|  0.0|
|[0.132862180406,0...|  0.0|
|[0.1533035231,0.1...|  0.0|
|[0.118001599962,0...|  0.0|
|[0.110992493712,0...|  0.0|
|[0.124299777916,0...|  0.0|
+--------------------+-----+
only showing top 20 rows



In [15]:
scaler = MinMaxScaler(inputCol='features', outputCol='scaledFeatures', min = 0, max=1000)
scalerModel = scaler.fit(output)
scaled_data= scalerModel.transform(output)

In [16]:
scaled_data.show()

+--------------------+-----+--------------------+
|            features|label|      scaledFeatures|
+--------------------+-----+--------------------+
|[0.136439587512,0...|  0.0|[519.818266700239...|
|[0.117038518483,0...|  0.0|[435.295463992565...|
|[0.0853077737447,...|  0.0|[297.057129121742...|
|[0.103049917216,0...|  0.0|[374.352647734368...|
|[0.151729948738,0...|  0.0|[586.432337466259...|
|[0.127046737192,0...|  0.0|[478.897323979332...|
|[0.123395302003,0...|  0.0|[462.989461602348...|
|[0.140027382431,0...|  0.0|[535.448873531282...|
|[0.117635200751,0...|  0.0|[437.894973202184...|
|[0.137400181488,0...|  0.0|[524.003195633498...|
|[0.148838734199,0...|  0.0|[573.836456502165...|
|[0.119846749928,0...|  0.0|[447.529820358620...|
|[0.0786813648231,...|  0.0|[268.188480022892...|
|[0.138335144235,0...|  0.0|[528.076459415282...|
|[0.101304207661,0...|  0.0|[366.747280005321...|
|[0.132862180406,0...|  0.0|[504.232915471732...|
|[0.1533035231,0.1...|  0.0|[593.287780078539...|


In [17]:
final_data=scaled_data.select('label', 'scaledFeatures')
final_data = final_data.withColumnRenamed('scaledFeatures', 'features')

In [18]:
train, test= final_data.randomSplit([0.7, 0.3])

In [19]:
train.count()

1599

In [20]:
test.count()

701

### Modeling

In [21]:
from pyspark.ml.classification import *
from pyspark.ml.evaluation import *
from pyspark.sql.functions import *
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [22]:
Bin_evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction')
MC_evaluator = MulticlassClassificationEvaluator(metricName='accuracy')

### Logit (Chi-Square Numfeatures=30)

In [23]:
classifier= LogisticRegression()

paramGrid = (ParamGridBuilder().addGrid(classifier.maxIter, [10, 15,20]).build())

crossval = CrossValidator(estimator=classifier,
                         estimatorParamMaps=paramGrid,
                         evaluator=MC_evaluator,
                         numFolds=2)

fitModel = crossval.fit(train)
BestModel = fitModel.bestModel

print('Intercept: ' +str(BestModel.interceptVector))
print('Coefficients: \n' + str(BestModel.coefficientMatrix))

LR_BestModel = BestModel

predictions=fitModel.transform(test)
accuracy = (MC_evaluator.evaluate(predictions))*100

print(accuracy)


Intercept: [-0.003439003624001725,-0.002283725172729804,0.0034400541631993136,0.04752550472044123,-0.11476264587359482,0.02029028946997164,0.09755982695268281,0.013719320256262548,-0.06896704095470169,0.017564674031312927,-0.02536440012795451,-0.13390175035615007,0.06876506095521794,0.0159320107295216,0.030938448647966882,-0.024071704514029048,0.06935017743125944,-0.10571034450194741,0.10790822349637036,-0.0067245575882144025,0.05645242834604122,-0.056281694167753,-0.007939152319171468]
Coefficients: 
DenseMatrix([[ 4.95209700e-04,  2.13343325e-03,  2.11590297e-03, ...,
              -2.62901034e-03, -1.18318864e-03,  1.66738739e-03],
             [-2.97965402e-04, -7.25974152e-04, -1.17296520e-04, ...,
               4.58680340e-03, -3.84027985e-03,  2.56502462e-03],
             [-2.61874111e-04, -5.87624342e-07,  1.46975057e-03, ...,
              -2.24086080e-03,  4.04777057e-04, -1.46199391e-03],
             ...,
             [ 4.39931340e-04, -1.28385417e-03, -9.23224224e-04, ..

In [24]:

coeff_array = BestModel.coefficientMatrix.toArray()
coeff_scores = []
for x in coeff_array[0]:
    coeff_scores.append(float(x))

result = spark.createDataFrame(zip(input_columns,coeff_scores), schema=['feature','coeff'])
result.show(100)

+--------------------+--------------------+
|             feature|               coeff|
+--------------------+--------------------+
|              1-ZCRm|4.952096998799204E-4|
|           2-Energym|0.002133433249608...|
|    3-EnergyEntropym|0.002115902972941641|
| 4-SpectralCentroidm|9.170826685377883E-4|
|   5-SpectralSpreadm|0.001575963647193...|
|  6-SpectralEntropym|9.582191454134053E-4|
|     7-SpectralFluxm|-0.00257449736591...|
|  8-SpectralRolloffm|2.296121039584361E-4|
|           9-MFCCs1m|0.004184343260159134|
|          10-MFCCs2m|-0.00169751164570...|
|          11-MFCCs3m|-7.45595017365970...|
|          12-MFCCs4m| 3.05928781998889E-4|
|          13-MFCCs5m| 8.27504010013759E-4|
|          14-MFCCs6m|-6.16418501499167...|
|          15-MFCCs7m|7.674519556212843E-5|
|          16-MFCCs8m|-1.69489856322324...|
|          17-MFCCs9m|-2.59460357055323...|
|         18-MFCCs10m|7.142146588992483E-4|
|         19-MFCCs11m|-6.96351996168954...|
|         20-MFCCs12m|2.66734048

### One Vs Rest Classifier fwe

In [25]:
lr = LogisticRegression()
classifier = OneVsRest(classifier = lr)

paramGrid = ParamGridBuilder()\
    .addGrid(lr.regParam,[0.1,0.01]) \
    .build()

crossval=CrossValidator(estimator=classifier,
                       estimatorParamMaps=paramGrid,
                       evaluator=MulticlassClassificationEvaluator(),
                       numFolds=2)

fitModel=crossval.fit(train)

BestModel = fitModel.bestModel

models= BestModel.models
for model in models:
    print('\033[1m' + 'Intercept: '+ '\033[0m',model.intercept,'\033[1m' + '\nCoefficients:'+ '\033[0m',model.coefficients)

predictions=fitModel.transform(test)
accuracy=(MC_evaluator.evaluate(predictions))*100
print(accuracy)

Intercept:  -2.7999517171778154 
Coefficients: [-0.00037696644266538396,0.0011226816683962402,0.000970664917348961,-0.0001613255933980758,0.0018469166285489556,0.00041010286510083474,-0.0021163221719274834,-0.0011238726434582421,0.004161335796431175,-0.001373365807902524,-0.00011800709386278434,0.0008813192867045943,0.0014691955886382737,-0.001732403125016051,0.00018969598961625993,-0.00020071333632668296,-0.0015203306876320124,0.000599917039894108,-0.000735786657309452,0.0008957802047652242,-0.0012495385993959568,0.0008351362170080642,-0.0028901048862312413,0.0012956210140090303,-0.0003582096448174655,-0.0003588605520001742,0.0014904554546084979,-0.002063375372801311,-0.0025266658824763316,0.0002438196659977831,-0.00011279535379109617,-0.0004517241535232548,-0.00010589711914763778,-0.001807076665276051,-0.0017889883508377944,-0.0001655284735695604,0.0005546572185582735,-0.0021156212544681716,-0.0018810625480938732,0.00016645264906517494,-0.0007927130615516953,-0.0003778194117554404,0.

48.502139800285306


### Multilayer Perceptron Classifier fwe

In [26]:
features = train.select(['features']).collect()
features_count = len(features[0][0])
class_count=final_data.select(countDistinct('label')).collect()
classes = class_count[0][0]

layers = [features_count, features_count+1, features_count, classes]

classifier = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed= 1234)

fitModel = classifier.fit(train)

print('\033[1m' + "Model Weights: "+ '\033[0m',fitModel.weights.size)


predictions = fitModel.transform(test)

accuracy = (MC_evaluator.evaluate(predictions))*100

print("Accuracy: ",accuracy)

Model Weights:  12023
Accuracy:  17.261055634807416


### NaiveBayes

In [27]:
classifier= NaiveBayes()
paramGrid = (ParamGridBuilder()\
             .addGrid(classifier.smoothing, [0.0,0.2,0.4,0.6])\
             .build())
crossval = CrossValidator(estimator=classifier,
                         estimatorParamMaps = paramGrid,
                         evaluator= MulticlassClassificationEvaluator(),
                         numFolds=2)

fitModel = crossval.fit(train)

predictions = fitModel.transform(test)
accuracy = (MC_evaluator.evaluate(predictions))*100
print("Accuracy: ",accuracy)

Accuracy:  42.51069900142653


### Decision Tree

In [28]:
classifier = DecisionTreeClassifier()
paramGrid = (ParamGridBuilder()\
            .addGrid(classifier.maxDepth, [2, 5, 10, 20, 30])\
            .addGrid(classifier.maxBins, [10,20,40,80, 100])\
            .build())

crossval = CrossValidator(estimator = classifier,
                         estimatorParamMaps=paramGrid,
                         evaluator=MulticlassClassificationEvaluator(),
                         numFolds=2)

fitModel = crossval.fit(train)

BestModel = fitModel.bestModel
featureImportances= BestModel.featureImportances.toArray()

print("Feature Importances: ", featureImportances)

predictions= fitModel.transform(test)
accuracy = (MC_evaluator.evaluate(predictions))*100
print("Accuracy: ", accuracy)

Feature Importances:  [0.13402499 0.04083916 0.04873417 0.03206116 0.01766315 0.0181274
 0.03989303 0.00098255 0.02673988 0.01608156 0.00611119 0.01645473
 0.01980442 0.00784832 0.01674616 0.01536611 0.01425441 0.00117906
 0.00533559 0.01352902 0.00496768 0.008035   0.         0.00849939
 0.00600319 0.00664964 0.00680157 0.00572981 0.00725704 0.
 0.00952587 0.00525596 0.00447276 0.         0.00370129 0.00359977
 0.01332449 0.00635981 0.01206405 0.00645195 0.00882531 0.
 0.01722819 0.02682398 0.00526288 0.00352552 0.00831006 0.00538072
 0.00624106 0.0026601  0.00715848 0.01929574 0.00275324 0.00970911
 0.01687144 0.009841   0.         0.00580747 0.00450015 0.00790248
 0.00189855 0.00467485 0.00494591 0.00175396 0.00125493 0.00665648
 0.00758963 0.00368669 0.00698258 0.03596477 0.1460194 ]
Accuracy:  37.660485021398


In [29]:
imp_scores = []
for x in featureImportances:
    imp_scores.append(int(x))

result = spark.createDataFrame(zip(input_columns,imp_scores), schema=['feature','score'])
print(result.orderBy(result["score"].desc()).show(truncate=False))

+----------------------+-----+
|feature               |score|
+----------------------+-----+
|35-ZCRstd             |0    |
|36-Energystd          |0    |
|37-EnergyEntropystd   |0    |
|38-SpectralCentroidstd|0    |
|39-SpectralSpreadstd  |0    |
|40-SpectralEntropystd |0    |
|41-SpectralFluxstd    |0    |
|42-SpectralRolloffstd |0    |
|43-MFCCs1std          |0    |
|44-MFCCs2std          |0    |
|45-MFCCs3std          |0    |
|46-MFCCs4std          |0    |
|47-MFCCs5std          |0    |
|48-MFCCs6std          |0    |
|49-MFCCs7std          |0    |
|50-MFCCs8std          |0    |
|51-MFCCs9std          |0    |
|18-MFCCs10m           |0    |
|19-MFCCs11m           |0    |
|20-MFCCs12m           |0    |
+----------------------+-----+
only showing top 20 rows

None


### Random Forest Classifer

In [30]:
classifier = RandomForestClassifier()
paramGrid = (ParamGridBuilder()\
            .addGrid(classifier.maxDepth, [2, 5, 10])
            .addGrid(classifier.maxBins, [5, 10 , 20])
            .addGrid(classifier.numTrees,[5, 20, 50])
            .build())

crossval = CrossValidator(estimator=classifier,
                         estimatorParamMaps=paramGrid,
                         evaluator=MulticlassClassificationEvaluator(),
                         numFolds=2)

fitModel = crossval.fit(train)

BestModel= fitModel.bestModel
featureImportances= BestModel.featureImportances.toArray()
print("Feature Importances: ", featureImportances)

predictions = fitModel.transform(test)

accuracy = (MC_evaluator.evaluate(predictions))*100

print(" ")
print("Accuracy: ", accuracy)

Feature Importances:  [0.02573566 0.02356898 0.01991815 0.01452942 0.01045946 0.00993903
 0.01668074 0.0089076  0.01559455 0.01588221 0.01371953 0.00998789
 0.00928669 0.00988818 0.00932718 0.00961034 0.00872837 0.00906725
 0.0085483  0.00907796 0.00963562 0.01085105 0.0086788  0.0127838
 0.00919309 0.00980125 0.01160123 0.01039926 0.00800358 0.00768724
 0.01053732 0.01035535 0.00904936 0.0114416  0.01082462 0.01236032
 0.01508101 0.01732041 0.02350336 0.01102697 0.01417001 0.01085671
 0.0181281  0.01957244 0.01359008 0.01181207 0.01383113 0.01825955
 0.01275283 0.01482237 0.0139311  0.01938239 0.01453853 0.01694109
 0.01555292 0.00928776 0.00884947 0.01093804 0.00779342 0.01120628
 0.00857455 0.01159077 0.01036301 0.00844019 0.00995531 0.01144762
 0.00887084 0.01103265 0.04715108 0.02612459 0.08164037]
 
Accuracy:  51.06990014265336


In [31]:
imp_scores = []
for x in featureImportances:
    imp_scores.append(float(x))
    
# Then zip with input_columns list and create a df
result = spark.createDataFrame(zip(input_columns,imp_scores), schema=['feature','score'])
print(result.orderBy(result["score"].desc()).show(truncate=False))

+----------------------+--------------------+
|feature               |score               |
+----------------------+--------------------+
|71-BPMessentia        |0.0816403681969719  |
|69-BPM                |0.04715108480109114 |
|70-BPMconf            |0.026124588351273806|
|1-ZCRm                |0.02573566485281646 |
|2-Energym             |0.023568979928749983|
|39-SpectralSpreadstd  |0.023503362622536356|
|3-EnergyEntropym      |0.019918148212840315|
|44-MFCCs2std          |0.01957243543323125 |
|52-MFCCs10std         |0.01938238864113405 |
|48-MFCCs6std          |0.018259551052789955|
|43-MFCCs1std          |0.018128100694104712|
|38-SpectralCentroidstd|0.01732041182293242 |
|54-MFCCs12std         |0.01694109371196689 |
|7-SpectralFluxm       |0.016680741605466416|
|10-MFCCs2m            |0.015882209320895402|
|9-MFCCs1m             |0.01559454820902609 |
|55-MFCCs13std         |0.015552920180658077|
|37-EnergyEntropystd   |0.01508101362848201 |
|50-MFCCs8std          |0.01482237

### Feature Selection

In [ ]:
from pyspark.ml.feature import ChiSqSelector

In [80]:


selector=ChiSqSelector(fwe=0.05, featuresCol="features", outputCol="selectedFeatures", labelCol= "label")
model=selector.fit(train)
result=model.transform(train)


In [81]:
train = result.select('label','selectedFeatures').withColumnRenamed('selectedFeatures', 'features')

In [82]:
new_test= model.transform(test)
test=new_test.select('label','selectedFeatures').withColumnRenamed('selectedFeatures', 'features')

In [83]:
train.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[174.433102127938...|
|  0.0|[268.188480022892...|
|  0.0|[278.771440597300...|
|  0.0|[312.108949814051...|
|  0.0|[319.529220420367...|
|  0.0|[337.143704291778...|
|  0.0|[350.856930326534...|
|  0.0|[355.801234298552...|
|  0.0|[357.765517922918...|
|  0.0|[359.825601229593...|
|  0.0|[366.747280005321...|
|  0.0|[374.352647734368...|
|  0.0|[376.623923825579...|
|  0.0|[380.752326609337...|
|  0.0|[402.720971071055...|
|  0.0|[407.322910385764...|
|  0.0|[408.955319636547...|
|  0.0|[419.899631408692...|
|  0.0|[431.815464667238...|
|  0.0|[437.414240861686...|
+-----+--------------------+
only showing top 20 rows



In [84]:
test.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[263.169877111654...|
|  0.0|[297.057129121742...|
|  0.0|[344.472423346100...|
|  0.0|[360.257783507319...|
|  0.0|[375.814351487179...|
|  0.0|[380.876129276615...|
|  0.0|[392.551986501107...|
|  0.0|[405.113188842400...|
|  0.0|[421.768375442709...|
|  0.0|[435.295463992565...|
|  0.0|[438.824880345582...|
|  0.0|[439.491229994906...|
|  0.0|[449.899410335232...|
|  0.0|[462.057387043392...|
|  0.0|[468.052453142057...|
|  0.0|[481.158976756092...|
|  0.0|[492.437503710276...|
|  0.0|[495.497024791874...|
|  0.0|[509.046741140820...|
|  0.0|[509.098065494723...|
+-----+--------------------+
only showing top 20 rows

